In [ ]:
import py_vncorenlp

# Tải mô hình về thư mục cụ thể (ví dụ: C:\vncorenlp)
py_vncorenlp.download_model(save_dir=r"C:\Users\Administrator\Documents\KeyWord_Trending\vncorenlp")

In [ ]:
import py_vncorenlp
model = py_vncorenlp.VnCoreNLP(save_dir=r'/home/chiennd/Desktop/khailq/vncorenlp')

In [ ]:
text_test = "Mỹ áp dụng thuế quan với các nước châu á như Myanmar, Trung Quốc, Nhật Bản"
annotated_test = model.annotate_text(text_test)
model.print_out(annotated_test)
print("\n")

In [ ]:
import json
from tqdm import tqdm
import os
import re
# Thêm thư viện để chia chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

# --- Cấu hình ---
INPUT_FILE_PATH = r'/home/chiennd/Desktop/khailq/data/test3.ndjson'
OUTPUT_FILE_PATH = r'/home/chiennd/Desktop/khailq/result/olddata_keywords_in2609_chunked.ndjson'

PRINT_KEYWORDS_PER_ARTICLE = True # Đặt là True để in keywords của từng bài
# THAY ĐỔI: Thêm cấu hình để log các bài báo quá lớn
LOG_OVERSIZED_ARTICLES = True # Đặt là True để in cảnh báo khi bài báo có > 3 chunks


def clean_text(text):
    if not text:
        return ""
    text = re.sub(r'[-–—]+', ' ', text)
    text = ' '.join(text.split())
    return text

def load_stopwords(file_path):
    stopwords = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            word = line.strip().strip('\"')
            if word:
                stopwords.append(word.lower())
    return tuple(stopwords)

BLACKLISTED_START_WORDS = load_stopwords(r"/home/chiennd/Desktop/khailq/blacklist/blacklist_keywords.txt")

CHUNK_GRAMMAR = [
    ("LEGAL_DOC_RULE_2", r"(?:<Np>|<N>)(?:\s+<N>)*(?:\s+<M>)+(?:\s+(?:<Np>|<N>|<Ny>))+"),
    ("LEGAL_DOC_RULE", r"(?:<Np>|<N>)(?:\s+<N>)*(\s+<M>)+"),
    ("NOUN_RULE", r"(?:<Np>|<N>|<Ny>)(?:\s+(?:<Np>|<N>|<Ny>))*"),
]

def compile_rules(rules):
    return [
        (name, re.compile(rule.replace("<", r"(?:\b").replace(">", r"\b)")))
        for name, rule in rules
    ]

COMPILED_CHUNK_RULES = compile_rules(CHUNK_GRAMMAR)

def is_valid_single_word_keyword(chunk_word, rule_name):
    tag = chunk_word.get('posTag')
    word_form = chunk_word.get('wordForm', '')
    if tag == 'Np' and len(word_form) > 5:
        return True
    if tag == 'Ny':
        if word_form.isupper() and len(word_form) >= 2:
            return True
    if rule_name == "VERB_RULE" and tag == 'V' and len(word_form) > 5:
        return True
    return False

def extract_keywords_2(annotated_data):
    seen_keywords_lower = set()
    ordered_keywords = []
    if not isinstance(annotated_data, dict):
        return []
    for sentence_words in annotated_data.values():
        if not sentence_words:
            continue
        pos_sequence = " ".join([word.get('posTag', '') for word in sentence_words])
        claimed_indices = [False] * len(sentence_words)
        for rule_name, rule_pattern in COMPILED_CHUNK_RULES:
            for match in rule_pattern.finditer(pos_sequence):
                start_char, end_char = match.span()
                start_word = len(pos_sequence[:start_char].strip().split()) if start_char > 0 else 0
                end_word = len(pos_sequence[:end_char].strip().split())
                if any(claimed_indices[i] for i in range(start_word, end_word)):
                    continue
                chunk_words = sentence_words[start_word:end_word]
                phrase_text = ' '.join(w.get('wordForm', '') for w in chunk_words).replace("_", " ")
                is_valid = False
                if len(chunk_words) > 1:
                    is_valid = True
                elif len(chunk_words) == 1:
                    if is_valid_single_word_keyword(chunk_words[0], rule_name):
                        is_valid = True
                if is_valid and phrase_text:
                    lower_phrase = phrase_text.lower()
                    if lower_phrase not in seen_keywords_lower:
                        is_blacklisted = False
                        for blacklisted_word in BLACKLISTED_START_WORDS:
                            if blacklisted_word in lower_phrase:
                                is_blacklisted = True
                                break
                        if is_blacklisted:
                            continue
                        seen_keywords_lower.add(lower_phrase)
                        ordered_keywords.append(phrase_text)
                        for i in range(start_word, end_word):
                            claimed_indices[i] = True
    return ordered_keywords


# --- Hàm xử lý chính 
def process_articles():
    """
    Đọc các bài báo từ file NDJSON, chia nhỏ văn bản, trích xuất keywords và lưu vào file mới.
    """
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50)

    try:
        with open(INPUT_FILE_PATH, 'r', encoding='utf-8') as infile, \
             open(OUTPUT_FILE_PATH, 'w', encoding='utf-8') as outfile:

            lines = infile.readlines()
            print(f"Bắt đầu xử lý {len(lines)} bài báo từ file '{INPUT_FILE_PATH}'...")

            for line in tqdm(lines, desc="Đang trích xuất keywords"):
                try:
                    article_data = json.loads(line)
                    content = article_data.get('content', '')
                    title = article_data.get('title', '')
                    combined_text = f"{title}. {content}" if title else content
                    
                    keywords = []
                    if combined_text.strip() and combined_text.strip() != '.':
                        chunks = text_splitter.split_text(combined_text)
                        

                        original_chunk_count = len(chunks)
                        if original_chunk_count > 3:
                            if LOG_OVERSIZED_ARTICLES:
                                article_id = article_data.get('_id', 'Unknown ID')

                                tqdm.write(f"  Article ID: {article_id} has {original_chunk_count} chunks. Processing first chunk only.")
                            chunks = chunks[:1] # Chỉ xử lý chunk đầu tiên
                        
                        final_keywords_for_article = []
                        seen_keywords_for_article = set()
                        
                        for chunk in chunks:
                            cleaned_chunk = clean_text(chunk)
                            if not cleaned_chunk:
                                continue
                            
                            annotated_chunk = model.annotate_text(cleaned_chunk)
                            keywords_from_chunk = extract_keywords_2(annotated_chunk)
                            
                            for keyword in keywords_from_chunk:
                                lower_keyword = keyword.lower()
                                if lower_keyword not in seen_keywords_for_article:
                                    seen_keywords_for_article.add(lower_keyword)
                                    final_keywords_for_article.append(keyword)
                        
                        keywords = final_keywords_for_article

                    article_data['keywords'] = keywords
                    outfile.write(json.dumps(article_data, ensure_ascii=False) + '\n')

                    if PRINT_KEYWORDS_PER_ARTICLE:
                        article_id = article_data.get('_id', 'Unknown ID')
                        tqdm.write(f"✅ ID: {article_id} - Keywords ({len(keywords)}): {keywords}")

                except json.JSONDecodeError:
                    tqdm.write(f"⚠️ Bỏ qua dòng không hợp lệ: {line.strip()}")
                except Exception as e:
                    tqdm.write(f"❌ Đã xảy ra lỗi khi xử lý dòng: {line.strip()} - Lỗi: {e}")

            print(f"\\n🎉 Xử lý hoàn tất! Kết quả đã được lưu vào file '{OUTPUT_FILE_PATH}'.")

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file đầu vào '{INPUT_FILE_PATH}'. Vui lòng kiểm tra lại đường dẫn.")
    except Exception as e:
        print(f"Đã có lỗi không mong muốn xảy ra: {e}")

# --- Điểm bắt đầu thực thi ---
if __name__ == "__main__":
    print("Khởi động quy trình xử lý bài báo với VnCoreNLP...")
    process_articles()